In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
dfAdmissions = pd.read_csv("./data/ADMISSIONS.csv")

dfDiagnoses = pd.read_csv("./data/DIAGNOSES_ICD.csv")

dfTitle = pd.read_csv("./data/d_icd_diagnoses.csv")

df = pd.merge(dfAdmissions, dfDiagnoses, how="inner", on=['SUBJECT_ID'])

dfWithTtitle = pd.merge(df, dfTitle, how="left", on=['ICD9_CODE'])

In [3]:
#creating vector of HAI related to pneumonia, and HAP

hcapCode = ["48230","48231","48232","48233", "48234", "48235", "48236", "48237", "48238", 
            "48239", "48240", "4830", "4822", "4820", "48240", "48241", "48242", "48243",
           "48244", "48245", "48246", "48247", "48248", "48249", "48282", "48281", "48283",
           "48284", "48289"]

#creating boolean values that are HAI related to pneumonia, and HAP
df['is_it_HAI'] = df['ICD9_CODE'].map(lambda x: any(x == i for i in hcapCode)) 

In [4]:
#selecting the relevant variables for logistic regression
dfFinal = df[['INSURANCE','LANGUAGE','RELIGION','MARITAL_STATUS','ETHNICITY','is_it_HAI']]

#drop na values
dfFinal = dfFinal.dropna()

In [101]:
#create dummy variables

dummy_insurance = pd.get_dummies(dfFinal['INSURANCE'], prefix='INSURANCE')
dummy_language = pd.get_dummies(dfFinal['LANGUAGE'], prefix = 'LANGUAGE')
dummy_religion = pd.get_dummies(dfFinal['RELIGION'], prefix = 'RELIGION')
dummy_maritalStatus = pd.get_dummies(dfFinal['MARITAL_STATUS'], prefix = 'MARITAL_STATUS')
dummy_ethnicity = pd.get_dummies(dfFinal['ETHNICITY'], prefix = 'ETHNICITY')

#create a new data frame to fit the dummy variables
colsToKeep = ['is_it_HAI']
data = dfFinal[colsToKeep].join(dummy_insurance.ix[:, 'INSURANCE_Medicaid':])
data = data.join(dummy_language.ix[:, 'LANGUAGE_* FU':])
data = data.join(dummy_religion.ix[:, 'RELIGION_BAPTIST':])
data = data.join(dummy_maritalStatus.ix[:, 'MARITAL_STATUS_LIFE PARTNER':])
data = data.join(dummy_ethnicity.ix[:, 'ETHNICITY_AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE':])

In [100]:
#

True

In [85]:
#logistic regression

from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()

train_cols = data.columns[1:]

logitModel = logistic.fit(data[train_cols], data['is_it_HAI'])



In [87]:
# examine the coefficients

for i in range(138):
    print(list(train_cols)[i], logitModel.coef_[0][i])

INSURANCE_Medicaid 0.389365642157
INSURANCE_Medicare 0.239442967801
INSURANCE_Private 0.115903944306
INSURANCE_Self Pay -0.079745453864
LANGUAGE_* FU 0.898709275895
LANGUAGE_** T -0.0593143008898
LANGUAGE_**SH -0.340806430029
LANGUAGE_**TO -0.43590454429
LANGUAGE_*AMH -0.0168175581014
LANGUAGE_*ARA -0.0200139740751
LANGUAGE_*ARM 0.409120563408
LANGUAGE_*BEN -0.294031674187
LANGUAGE_*BOS -0.0182444142937
LANGUAGE_*BUL -0.400477486328
LANGUAGE_*BUR -0.567960703646
LANGUAGE_*CAN -0.246368265223
LANGUAGE_*CDI -0.155649627844
LANGUAGE_*CHI 1.48875346471
LANGUAGE_*CRE -0.0116915592681
LANGUAGE_*DEA -0.0688876918263
LANGUAGE_*DUT -0.0709473045152
LANGUAGE_*FAR -0.205187280335
LANGUAGE_*FIL -0.110107195927
LANGUAGE_*FUL -0.120614350442
LANGUAGE_*GUJ -0.361714690305
LANGUAGE_*HUN -1.23882525017
LANGUAGE_*IBO -0.173788978251
LANGUAGE_*KHM -0.104837711882
LANGUAGE_*LEB -0.0450469331245
LANGUAGE_*LIT -0.0495212359686
LANGUAGE_*MAN 1.48089585224
LANGUAGE_*MOR -0.534204074882
LANGUAGE_*NEP -0.086179

In [105]:
#using statsmodel library

data['intercept'] = 1.0

train_cols = data.columns[1:]

logit = sm.Logit(data['is_it_HAI'], data[train_cols])


In [106]:

result = logit.fit()



         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix